In [2]:
import pandas as pd
from baseline_utils import get_data_for_NN, set_seed
import os

In [3]:
set_seed(420)

In [16]:
def compute_accuracy(y, yhat):
    correct = sum(1 for yt, yh in zip(y, yhat) if yt == yh)
    return correct / len(y)

def compute_recall(y, yhat):
    true_positives = sum(1 for yt, yh in zip(y, yhat) if yt == 1 and yh == 1)
    false_negatives = sum(1 for yt, yh in zip(y, yhat) if yt == 1 and yh == 0)
    # Avoid division by zero
    if true_positives + false_negatives == 0:
        return 0.0
    return true_positives / (true_positives + false_negatives)

# Label length weirdness
While comparing actual and predicted labels, we came across the weird problem, that the labels did not match the original text inputs anymore. To be exact, the labels were suddenly more numerous than the tokens we had them for. Here is the reason: The tokeniser splits some words that we would otherwise not think to split (of row 426 of the evaluation data, i.e. after overflow handling):


In [4]:
tokens = ['[CLS]', '[CLS]', 'In', 'welcher', 'Weise', 'wurde', 'die', 'Red', 'Bull', 'Arena', 'in', 'W', '##als', '-', 'Sie', '##zen', '##heim', 'für', 'die', 'Fu', '##ss', '##ball', '-', 'EM', 'umgebaut', '?', '[SEP]', 'der', 'Red', '-', 'Bull', '-', 'Arena', 'werden', 'für', 'der', 'Turnier', 'in', 'zwei', 'Phase', 'um', '##bauen', '.', 'in', 'der', 'erst', 'Phase', 'werden', 'der', 'Stadion', 'für', 'ein', 'Kap', '##azi', '##tät', 'von', '30', '.', '000', 'Zuschauer', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
print("Number of tokens after the [SEP] token:", len(tokens[tokens.index('[SEP]')+1:]))

Number of tokens after the [SEP] token: 229


In [5]:
y="""0    0    0    0    0    0    0    0    0    0    0    1    1    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100 -100
 -100 -100 -100 -100 -100""".split()
len(y)

229

In [6]:
yhat="""0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 1 1 1
 0 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1
 0 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1""".split()
len(yhat)

229

These are the input tokens after tokenisation. The word „Fußball“ has been split into 3 tokens, at positions 19, 20, 21


In [7]:
print(tokens[19:22])
print(y[19:22])
print(yhat[19:22])

['Fu', '##ss', '##ball']
['0', '0', '0']
['0', '1', '0']


Curiously, they have different labels, according to our model at least. Originally, of course, theyre identical.
Here is another example for Wals-Siezenheim. Wals was apparently false, while Siezenheim was correct. Both town-names were split.

In [8]:
print(tokens[11:17])
print(y[11:17])
print(yhat[11:17])

['W', '##als', '-', 'Sie', '##zen', '##heim']
['1', '1', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '1']


In [9]:
datapath = "../data/output/val_predictions_mbert2.csv"
with open(datapath, "r") as f:
    raw_data = f.readlines()
raw_data = "".join(raw_data).replace("\n", "")[1:-5].split("[END]")
raw_data[0]

"'[CLS]', '¿', 'El', 'municipio', 'H', '##ü', '##ffel', '##sheim', ',', 'está', 'ubicado', 'en', 'el', 'este', 'o', 'el', 'oeste', 'de', 'Alemania', '?', '[SEP]', 'h', '##ü', '##ffel', '##sheim', 'ser', 'uno', 'municipio', 'ubicado', 'en', 'el', 'estado', 'de', 'bavi', '##era', ',', 'que', 'estar', 'en', 'el', 'oeste', 'de', 'Alemania', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [10]:
data_lists = [row.split("[SEP]") for row in raw_data]
data_lists[1]

["['[CLS]', '¿', 'En', 'qué', 'país', 'se', 'encuentra', 'la', 'ciudad', 'de', 'Tor', '##u', '##ń', '?', '",
 "', 'el', 'ciudad', 'de', 'to', '##ru', '##ń', 'él', 'encontrar', 'en', 'el', 'país', 'de', 'polo', '##nia', '.', '",
 "', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[

each element is now a list of 3 items: query, response, padding, labels, predictions

In [14]:
data_dicts = [{
    "query": row[0].replace("', '", " ").strip().strip("[").strip("]").strip("'").strip("[CLS] "),
    "response": row[1].replace("', '", " ").strip().strip("[").strip("]").strip("'"),
    "labels": [int(lab) for lab in row[3][1:-1].split()],
    "predictions": [int(lab) for lab in row[4][1:-1].split()]
} for row in data_lists]
data_dicts[0]

{'query': '¿ El municipio H ##ü ##ffel ##sheim , está ubicado en el este o el oeste de Alemania ?',
 'response': 'h ##ü ##ffel ##sheim ser uno municipio ubicado en el estado de bavi ##era , que estar en el oeste de Alemania .',
 'labels': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 'predictions': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0]}

In [15]:
df = pd.DataFrame(data_dicts)
df

,query,response,labels,predictions
0,"¿ El municipio H ##ü ##ffel ##sheim , está ubi...",h ##ü ##ffel ##sheim ser uno municipio ubicado...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, ..."
1,¿ En qué país se encuentra la ciudad de Tor ##...,el ciudad de to ##ru ##ń él encontrar en el pa...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,¿ En qué estado de Estados Unidos se encuentra...,le ##wis ##town él encontrar en el estado de m...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
3,When did Je ##f Ras ##kin die ?,"Je ##f Ra ##kin die on March 1 , 2011 . he be ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0]","[1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,玳 瑁 蜗 牛 属 的 物 种 分 布 在 哪 些 地 区 ？,玳 瑁 螺 （ 学 名 ： Arch ##acha ##tina margin ##ata ...,"[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
...,...,...,...,...
494,Wie viele Zuschauer haben Platz im Gill ##ette...,der Gil ##ette Stadium sein mit ein Kap ##azi ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, ..."
495,Hu ##r många år har Chris Clare ##mont sk ##ri...,den finn ##as ingen känd upp ##ska ##ttning av...,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
496,金 属 键 决 定 了 金 属 的 哪 些 物 理 性 质 ？,金 属 晶 体 中 ， 金 属 原 子 之 间 通 过 金 属 结 合 力 （ 金 属 离 ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ..."
497,Gegen welche ##n Verein verlor Hamilton Academ...,Hamilton Academic ver ##liere ##n in der Play ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [13]:
df.iloc[110]

query          [CLS] Welch ##e Rolle spielte Ted Lange in der...
response       A : Ted Lange spielen der Rolle der Bord ##arz...
labels         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...
predictions    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...
Name: 110, dtype: object

In [17]:
df['accuracy'] = df.apply(lambda row: compute_accuracy(row['labels'], row['predictions']), axis=1)
df['recall'] = df.apply(lambda row: compute_recall(row['labels'], row['predictions']), axis=1)
df

,query,response,labels,predictions,accuracy,recall
0,"¿ El municipio H ##ü ##ffel ##sheim , está ubi...",h ##ü ##ffel ##sheim ser uno municipio ubicado...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, ...",0.736842,0.500000
1,¿ En qué país se encuentra la ciudad de Tor ##...,el ciudad de to ##ru ##ń él encontrar en el pa...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.916667,0.000000
2,¿ En qué estado de Estados Unidos se encuentra...,le ##wis ##town él encontrar en el estado de m...,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0.666667,0.000000
3,When did Je ##f Ras ##kin die ?,"Je ##f Ra ##kin die on March 1 , 2011 . he be ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0]","[1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0.600000,0.400000
4,玳 瑁 蜗 牛 属 的 物 种 分 布 在 哪 些 地 区 ？,玳 瑁 螺 （ 学 名 ： Arch ##acha ##tina margin ##ata ...,"[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",0.494118,0.446809
...,...,...,...,...,...,...
494,Wie viele Zuschauer haben Platz im Gill ##ette...,der Gil ##ette Stadium sein mit ein Kap ##azi ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, ...",0.833333,0.857143
495,Hu ##r många år har Chris Clare ##mont sk ##ri...,den finn ##as ingen känd upp ##ska ##ttning av...,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.176471,1.000000
496,金 属 键 决 定 了 金 属 的 哪 些 物 理 性 质 ？,金 属 晶 体 中 ， 金 属 原 子 之 间 通 过 金 属 结 合 力 （ 金 属 离 ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...",0.466667,0.000000
497,Gegen welche ##n Verein verlor Hamilton Academ...,Hamilton Academic ver ##liere ##n in der Play ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.631579,0.000000


# Ideas to go on:

- Try other tokenisers, e.g. using a language specific one for each observation
- Thus far we havent lemmatized the query before training, maybe that would help
- Try different max_lens
- Try different batch sizes